## Bribes research

In [16]:
import math

import pandas as pd
import requests
from IPython.core.display_functions import display

from notebooks.bribes_research.constants import BRIBES_URL
from notebooks.bribes_research.constants import DASHBOARD_URL
from notebooks.bribes_research.constants import PROTOCOL_AURA
from notebooks.bribes_research.constants import PROTOCOL_CVX
from notebooks.bribes_research.constants import TABLE_COLUMNS
from notebooks.bribes_research.helpers import calculate_metrics
from notebooks.bribes_research.helpers import fetch_token_prices

pd.set_option('display.max_rows', 500)
BADGER_BRIBE = 40000
DELTA = 20000
STEP = 2000
BADGER_LOW_BOUND = BADGER_BRIBE - DELTA
BADGER_UPPER_BOUND = BADGER_BRIBE + DELTA

table_rows = []
token_prices = fetch_token_prices(['badger-dao'])
badger_price = token_prices['badger-dao']['usd']

cvx_bribes = requests.post(BRIBES_URL, json={'platform': "votium", 'protocol': "cvx-crv"}).json()
cvx_dash = requests.post(DASHBOARD_URL, json={"id": "bribes-overview-votium"}).json()
aura_bribes = requests.post(BRIBES_URL, json={'platform': "hh", 'protocol': "aura-bal"}).json()
aura_dash = requests.post(DASHBOARD_URL, json={"id": "bribes-overview-aura"}).json()
total_bribes_cvx = sum([bribe['amountDollars'] for bribe in cvx_bribes['epoch']['bribes']])
dollar_spent_on_bribes_cvx = cvx_dash['dashboard']['rewardPerDollarBribe']

total_bribes_aura = sum([bribe['amountDollars'] for bribe in aura_bribes['epoch']['bribes']])
dollar_spent_on_bribes_aura = aura_dash['dashboard']['rewardPerDollarBribe']
for bribe in range(BADGER_LOW_BOUND, BADGER_UPPER_BOUND, STEP):
    badger_bribe_in_dollar = bribe * badger_price
    # cvx_dilution_100, cvx_in_out_100 = calculate_metrics(badger_bribe_in_dollar, total_bribes_cvx,
    #                                                      dollar_spent_on_bribes_cvx)
    # table_rows.append(
    #     [PROTOCOL_CVX, badger_bribe_in_dollar, (badger_bribe_in_dollar / badger_bribe_in_dollar) * 100,
    #      badger_bribe_in_dollar, cvx_dilution_100, cvx_in_out_100]
    # )
    # aura_dilution_100, aura_in_out_100 = calculate_metrics(badger_bribe_in_dollar, total_bribes_aura,
    #                                                        dollar_spent_on_bribes_aura)
    # table_rows.append(
    #     [PROTOCOL_AURA, badger_bribe_in_dollar, (badger_bribe_in_dollar / badger_bribe_in_dollar) * 100,
    #      badger_bribe_in_dollar, aura_dilution_100, aura_in_out_100]
    # )
    cvx_dilution_50, cvx_in_out_50 = calculate_metrics(badger_bribe_in_dollar / 2, total_bribes_cvx,
                                                       dollar_spent_on_bribes_cvx)
    aura_dilution_50, aura_in_out_50 = calculate_metrics(badger_bribe_in_dollar / 2, total_bribes_aura,
                                                         dollar_spent_on_bribes_aura)
    # table_rows.append(
    #     [PROTOCOL_AURA, badger_bribe_in_dollar, ((badger_bribe_in_dollar / 2) / badger_bribe_in_dollar) * 100,
    #      badger_bribe_in_dollar / 2, aura_dilution_50, aura_in_out_50]
    # )

    # Rebalance here
    cvx_in_out_floating = cvx_in_out_50
    aura_in_out_floating = aura_in_out_50
    rebalancing_step = 0.01  # in %
    cvx_bribe_portion = aura_bribe_portion = badger_bribe_in_dollar / 2
    cvx_dilution = None
    aura_dilution = None
    while not math.isclose(cvx_in_out_floating, aura_in_out_floating, abs_tol=0.5):
        if cvx_in_out_floating > aura_in_out_floating:
            # Take bribe portion from aura and give it to cvx
            bribe_stolen = aura_bribe_portion * rebalancing_step
            aura_bribe_portion -= bribe_stolen
            cvx_bribe_portion += bribe_stolen
        elif aura_in_out_floating > cvx_in_out_floating:
            # Take bribe portion from cvx and give it to aura
            bribe_stolen = aura_bribe_portion * rebalancing_step
            cvx_bribe_portion -= bribe_stolen
            aura_bribe_portion += bribe_stolen
        if cvx_bribe_portion <= 0 or aura_bribe_portion <= 0:
            break
        cvx_dilution, cvx_in_out_floating = calculate_metrics(cvx_bribe_portion, total_bribes_cvx,
                                                              dollar_spent_on_bribes_cvx)
        aura_dilution, aura_in_out_floating = calculate_metrics(aura_bribe_portion, total_bribes_aura,
                                                                dollar_spent_on_bribes_aura)

    table_rows.append(
        [PROTOCOL_CVX, badger_bribe_in_dollar, round(cvx_bribe_portion / badger_bribe_in_dollar * 100, 2),
         cvx_bribe_portion, cvx_dilution, cvx_in_out_floating]
    )
    table_rows.append(
        [PROTOCOL_AURA, badger_bribe_in_dollar, round(aura_bribe_portion / badger_bribe_in_dollar * 100, 2),
         aura_bribe_portion, aura_dilution, aura_in_out_floating]
    )
dataframe = pd.DataFrame(data=table_rows, columns=TABLE_COLUMNS)
display(dataframe)

,Protocol,BADGER total bribe in $,Bribes Portion %,Bribes Portion,Dilution,$I/O
0,CVX,85400.0,-0.34,-288.795830,0.020767,149.510989
1,AURA,85400.0,100.34,85688.795830,27.047464,185.478033
2,CVX,93940.0,-0.34,-317.675414,0.022844,149.507883
3,AURA,93940.0,100.34,94257.675414,29.752210,178.601355
4,CVX,102480.0,-0.34,-346.554997,0.024920,149.504778
5,AURA,102480.0,100.34,102826.554997,32.456957,171.724678
6,CVX,111020.0,-0.34,-375.434580,0.026997,149.501672
7,AURA,111020.0,100.34,111395.434580,35.161703,164.848001
8,CVX,119560.0,-0.34,-404.314163,0.029074,149.498567
9,AURA,119560.0,100.34,119964.314163,37.866449,157.971324
